<a href="https://colab.research.google.com/github/rajkumarshedage/Health-Insurance-Cross-Sell-Prediction/blob/main/Health_Insurance_Cross_Sell_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Name - Health Insurance Cross Sell Prediction
## Project Type - Classification
## Contribution - Individual
### GitHub Link
https://github.com/rajkumarshedage/Health-Insurance-Cross-Sell-Prediction

### Problem Statement
Business Problem Overview
An insurance company is looking to build a model to predict whether their past health insurance policyholders would also be interested in purchasing vehicle insurance from the company.

An insurance policy is a contract where the company agrees to provide compensation for specific losses, injuries, or death in exchange for regular payments called premiums.

In this case, the company wants to use information about the customer's demographics, vehicle details, and previous insurance policy information to predict their interest in vehicle insurance.

This information would be used to target marketing efforts and optimize the company's revenue.

## Business Objective
The business objective for a health insurance cross-sell prediction is to identify potential customers who are likely to purchase additional products.

This allows the insurance company to target their marketing efforts towards these individuals, potentially increasing sales and revenue.

Additionally, by identifying and targeting high-risk customers, the insurance company can also improve their risk management and profitability.